In [1]:
import os

import numpy as np
import pandas as pd

## Compare with Baseline / Best-case (Oracle) Models

In [2]:
gen_artifacts_fpath = "/hpf/projects/lsung/projects/mimic4ds/Experiments/domain_gen/artifacts"
adapt_artifacts_fpath = "/hpf/projects/lsung/projects/mimic4ds/Experiments/domain_adapt/artifacts" 
base_artifacts_fpath = "/hpf/projects/lsung/projects/mimic4ds/Experiments/baseline/artifacts"
tables_fpath = '/hpf/projects/lsung/projects/mimic4ds/Experiments/domain_adapt/tables/'

In [3]:
methods = [
    ['erm','Domain Generalization',''],
    ['irm','Domain Generalization',''],
    ['dro','Domain Generalization',''],
    ['al_layer','Domain Generalization',''],
    ['coral','Domain Generalization',''],
    ['al_layer','Domain Adaptation','100'],
    ['coral','Domain Adaptation','100'],
    ['al_layer','Domain Adaptation','500'],
    ['coral','Domain Adaptation','500'],
    ['al_layer','Domain Adaptation','1000'],
    ['coral','Domain Adaptation','1000'],
    ['al_layer','Domain Adaptation','1500'],
    ['coral','Domain Adaptation','1500'],
]

In [4]:
alpha = 0.05

df_results = pd.DataFrame()
for analysis_id in ['mortality','longlos','invasivevent','sepsis']:
    for method in methods:
        for comparison_model in ['base','oracle']:

            fpath = os.path.join(
                gen_artifacts_fpath 
                if method[1]=='Domain Generalization' else 
                adapt_artifacts_fpath,
                f"analysis_id={analysis_id}",
                "results/compare_with_baseline_exp",
                f"nn_{method[0]}_{comparison_model}.csv"
                if method[1]=='Domain Generalization' else
                f"nn_{method[0]}_{method[2]}_{comparison_model}.csv"
            )

            df = pd.read_csv(fpath)

            lower_CI = df.groupby("metric").apply(lambda x: x.quantile(alpha/2))['performance_diff'].reset_index()
            lower_CI.rename(columns={'performance_diff':'CI_lower'}, inplace=True)

            upper_CI = df.groupby("metric").apply(lambda x: x.quantile(1-alpha/2))['performance_diff'].reset_index()
            upper_CI.rename(columns={'performance_diff':'CI_upper'}, inplace=True)
            
            med_CI = df.groupby("metric").apply(lambda x: x.quantile(0.5))['performance_diff'].reset_index()
            med_CI.rename(columns={'performance_diff':'CI_med'}, inplace=True)

            df_CIs = pd.merge(lower_CI, upper_CI, how='left', left_on='metric', right_on='metric')
            df_CIs = pd.merge(df_CIs, med_CI, how='left', left_on='metric', right_on='metric')
            df_CIs['sig'] = df_CIs['CI_lower']*df_CIs['CI_upper']>0
            df_CIs['analysis_id'] = analysis_id
            df_CIs['framework'] = method[1]
            df_CIs['comparison_model'] = comparison_model
            df_CIs['train_method'] = method[0]
            df_CIs['n_ood'] = method[2]

            df_results = pd.concat((df_results, df_CIs),axis=0)

# Calculate significane & clean columns
df_results['sig'] = df_results['CI_lower']*df_results['CI_upper']>0

df_results['sig'].replace({
        False:'',
        True:'*',
        np.nan:''
    }, inplace=True)

df_results['Difference'] = (
        df_results['CI_med'].apply('{:.3f}'.format) + 
        df_results['sig'] +
        " (" + 
        df_results['CI_lower'].apply('{:.3f}'.format) + 
        ',' + 
        df_results['CI_upper'].apply('{:.3f}'.format) +
        ')'
    )

df_results.drop(columns=['CI_med','CI_lower','CI_upper','sig'], inplace=True)

# rename columns and values
df_results.rename(columns = {
    'metric':'Metric',
    'analysis_id':'Task',
    'n_ood':'Unlabeled OOD Samples',
    'framework': 'Framework',
    'train_method':'Method'},inplace=True)

df_results['Metric'].replace(
    {
        'auc':"AUROC",
        'auprc':"AUPRC",
        'ace_abs_logistic_log':'Calibration'
    },
    inplace=True
)

df_results['Method'].replace(
    {
        'al_layer':"AL",
        'coral':"CORAL",
        'erm':"ERM",
        'irm':'IRM',
        'dro':'GroupDRO'
    },
    inplace=True
)

df_results['Task'].replace(
    {
        'longlos':'Long LOS',
        'sepsis':'Sepsis',
        'mortality':'Mortality',
        'invasivevent':'Invasive Ventilation',
    },
    inplace=True
)

# pivot tables
# compare with base
df_results_base = df_results.query(
    "comparison_model=='base' and Metric==['AUROC','AUPRC','Calibration']"
).pivot(
    index=["Task","Metric"],columns=["Framework","Unlabeled OOD Samples","Method"],values=["Difference"]
)
df_results_base.fillna(" ",inplace = True)
df_results_base.columns = pd.MultiIndex.from_tuples([x[1:] for x in df_results_base.columns], names = ['Framework','Unlabelled OOD Samples','Method'])
df_results_base = df_results_base.reindex(labels = ['Long LOS','Sepsis','Mortality','Invasive Ventilation'], level=0)
df_results_base = df_results_base.reindex(labels = ['AUROC','AUPRC','Calibration'],level=1)

# compare with oracle
df_results_oracle = df_results.query(
    "comparison_model=='oracle' and Metric==['AUROC','AUPRC','Calibration']"
).pivot(
    index=["Task","Metric"],columns=["Framework","Unlabeled OOD Samples","Method"],values=["Difference"]
)
df_results_oracle.fillna(" ",inplace = True)
df_results_oracle.columns = pd.MultiIndex.from_tuples([x[1:] for x in df_results_oracle.columns], names = ['Framework','Unlabelled OOD Samples','Method'])
df_results_oracle = df_results_oracle.reindex(labels = ['Long LOS','Sepsis','Mortality','Invasive Ventilation'], level=0)
df_results_oracle = df_results_oracle.reindex(labels = ['AUROC','AUPRC','Calibration'],level=1)

In [5]:
# results relative to ERM[08-10]
df_results_base

Framework                           Domain Generalization  \
Unlabelled OOD Samples                                      
Method                                                ERM   
Task                 Metric                                 
Long LOS             AUROC           0.034* (0.030,0.038)   
                     AUPRC           0.021* (0.016,0.026)   
                     Calibration  -0.028* (-0.030,-0.026)   
Sepsis               AUROC           0.049* (0.041,0.057)   
                     AUPRC           0.039* (0.034,0.045)   
                     Calibration  -0.034* (-0.036,-0.031)   
Mortality            AUROC           0.017* (0.013,0.021)   
                     AUPRC           0.077* (0.066,0.089)   
                     Calibration     0.003* (0.002,0.004)   
Invasive Ventilation AUROC           0.007* (0.002,0.012)   
                     AUPRC           0.007 (-0.004,0.018)   
                     Calibration  -0.006* (-0.008,-0.004)   

Framework                                                  \
Unlabelled OOD Samples                                      
Method                                                IRM   
Task                 Metric                                 
Long LOS             AUROC           0.034* (0.030,0.038)   
                     AUPRC           0.024* (0.019,0.029)   
                     Calibration  -0.035* (-0.037,-0.033)   
Sepsis               AUROC           0.045* (0.038,0.053)   
                     AUPRC           0.037* (0.032,0.043)   
                     Calibration  -0.033* (-0.036,-0.031)   
Mortality            AUROC           0.017* (0.013,0.022)   
                     AUPRC           0.079* (0.067,0.090)   
                     Calibration     0.002* (0.001,0.003)   
Invasive Ventilation AUROC           0.007* (0.002,0.011)   
                     AUPRC           0.006 (-0.005,0.017)   
                     Calibration  -0.006* (-0.008,-0.004)   

Framework                                                  \
Unlabelled OOD Samples                                      
Method                                           GroupDRO   
Task                 Metric                                 
Long LOS             AUROC           0.035* (0.031,0.039)   
                     AUPRC           0.022* (0.017,0.028)   
                     Calibration  -0.012* (-0.015,-0.010)   
Sepsis               AUROC           0.019* (0.013,0.024)   
                     AUPRC           0.013* (0.009,0.017)   
                     Calibration  -0.023* (-0.025,-0.021)   
Mortality            AUROC        -0.033* (-0.040,-0.026)   
                     AUPRC           0.035* (0.019,0.050)   
                     Calibration     0.029* (0.027,0.031)   
Invasive Ventilation AUROC           0.006* (0.001,0.011)   
                     AUPRC          -0.001 (-0.012,0.011)   
                     Calibration  -0.004* (-0.008,-0.001)   

Framework                                                  \
Unlabelled OOD Samples                                      
Method                                                 AL   
Task                 Metric                                 
Long LOS             AUROC           0.035* (0.031,0.038)   
                     AUPRC           0.023* (0.018,0.028)   
                     Calibration  -0.029* (-0.031,-0.027)   
Sepsis               AUROC           0.048* (0.041,0.056)   
                     AUPRC           0.038* (0.032,0.043)   
                     Calibration  -0.035* (-0.037,-0.032)   
Mortality            AUROC           0.016* (0.012,0.020)   
                     AUPRC           0.077* (0.065,0.089)   
                     Calibration     0.003* (0.002,0.004)   
Invasive Ventilation AUROC           0.006* (0.001,0.011)   
                     AUPRC           0.002 (-0.009,0.013)   
                     Calibration  -0.006* (-0.008,-0.004)   

Framework                                                  \
Unlabelled OOD Samples         

In [6]:
# results relative to ERM[17-19]
df_results_oracle

Framework                           Domain Generalization  \
Unlabelled OOD Samples                                      
Method                                                ERM   
Task                 Metric                                 
Long LOS             AUROC        -0.024* (-0.030,-0.019)   
                     AUPRC        -0.009* (-0.016,-0.002)   
                     Calibration     0.029* (0.024,0.033)   
Sepsis               AUROC        -0.041* (-0.049,-0.033)   
                     AUPRC        -0.104* (-0.118,-0.090)   
                     Calibration     0.018* (0.014,0.022)   
Mortality            AUROC        -0.009* (-0.013,-0.005)   
                     AUPRC           0.048* (0.036,0.059)   
                     Calibration     0.004* (0.003,0.005)   
Invasive Ventilation AUROC           0.005* (0.001,0.009)   
                     AUPRC           0.025* (0.016,0.035)   
                     Calibration    -0.001 (-0.003,0.001)   

Framework                                                  \
Unlabelled OOD Samples                                      
Method                                                IRM   
Task                 Metric                                 
Long LOS             AUROC        -0.024* (-0.029,-0.019)   
                     AUPRC          -0.006 (-0.013,0.000)   
                     Calibration     0.022* (0.016,0.027)   
Sepsis               AUROC        -0.045* (-0.053,-0.037)   
                     AUPRC        -0.106* (-0.120,-0.092)   
                     Calibration     0.019* (0.015,0.023)   
Mortality            AUROC        -0.008* (-0.012,-0.005)   
                     AUPRC           0.049* (0.038,0.061)   
                     Calibration     0.003* (0.002,0.004)   
Invasive Ventilation AUROC           0.005* (0.000,0.009)   
                     AUPRC           0.024* (0.014,0.033)   
                     Calibration    -0.001 (-0.003,0.001)   

Framework                                                  \
Unlabelled OOD Samples                                      
Method                                           GroupDRO   
Task                 Metric                                 
Long LOS             AUROC        -0.023* (-0.028,-0.017)   
                     AUPRC        -0.008* (-0.015,-0.001)   
                     Calibration     0.044* (0.039,0.049)   
Sepsis               AUROC        -0.071* (-0.081,-0.062)   
                     AUPRC        -0.130* (-0.144,-0.116)   
                     Calibration     0.029* (0.025,0.033)   
Mortality            AUROC        -0.058* (-0.066,-0.051)   
                     AUPRC           0.005 (-0.010,0.021)   
                     Calibration     0.030* (0.028,0.032)   
Invasive Ventilation AUROC           0.004 (-0.001,0.008)   
                     AUPRC           0.017* (0.008,0.027)   
                     Calibration     0.000 (-0.001,0.002)   

Framework                                                  \
Unlabelled OOD Samples                                      
Method                                                 AL   
Task                 Metric                                 
Long LOS             AUROC        -0.023* (-0.029,-0.018)   
                     AUPRC        -0.007* (-0.014,-0.000)   
                     Calibration     0.028* (0.022,0.033)   
Sepsis               AUROC        -0.042* (-0.050,-0.034)   
                     AUPRC        -0.105* (-0.119,-0.092)   
                     Calibration     0.017* (0.013,0.021)   
Mortality            AUROC        -0.010* (-0.014,-0.006)   
                     AUPRC           0.048* (0.036,0.059)   
                     Calibration     0.004* (0.003,0.005)   
Invasive Ventilation AUROC           0.004 (-0.001,0.008)   
                     AUPRC           0.020* (0.010,0.030)   
                     Calibration    -0.002 (-0.004,0.000)   

Framework                                                  \
Unlabelled OOD Samples         

## compare with ERM Models

In [8]:
gen_results_fpath = '/hpf/projects/lsung/projects/mimic4ds/Experiments/domain_gen/results'
adapt_results_fpath = '/hpf/projects/lsung/projects/mimic4ds/Experiments/domain_adapt/results'

In [9]:
alpha = 0.05

results = {
    "gen":pd.read_csv(f"{gen_results_fpath}/model_comparison_{alpha}.csv").query("phase=='test' and `lambda`==-1 and group==1 and evaluation_method=='avg'").reset_index(drop=True),
    "adapt":pd.read_csv(f"{adapt_results_fpath}/model_comparison_{alpha}.csv").query("phase=='test' and `lambda`==-1 and group==1 and evaluation_method=='avg'").reset_index(drop=True),
}

# add additional columns
results['gen']['framework'] = 'Domain Generalization'
results['gen']['n_ood'] = ''
results['adapt']['framework'] = 'Domain Adaptation'

# combine & clean
df_results = pd.concat((results['gen'],results['adapt']),axis=0,ignore_index=True)
df_results['sig'] = df_results['ci_lower']*df_results['ci_upper']>0

df_results['sig'].replace({
        False:'',
        True:'*',
        np.nan:''
    }, inplace=True)

df_results['Difference'] = (
        df_results['ci_med'].apply('{:.3f}'.format) + 
        df_results['sig'] +
        " (" + 
        df_results['ci_lower'].apply('{:.3f}'.format) + 
        ',' + 
        df_results['ci_upper'].apply('{:.3f}'.format) +
        ')'
    )

df_results.drop(
    columns=['phase','evaluation_method','lambda','group','Unnamed: 0', 'sig','ci_med','ci_lower','ci_upper'],
    inplace=True
)

# rename columns and values
df_results.rename(columns = {
    'metric':'Metric',
    'analysis_id':'Task',
    'n_ood':'Unlabeled OOD Samples',
    'framework': 'Framework',
    'train_method':'Method'},inplace=True)

df_results['Metric'].replace(
    {
        'auc':"AUROC",
        'auprc':"AUPRC",
        'ace_abs_logistic_log':'Calibration'
    },
    inplace=True
)

df_results['Method'].replace(
    {
        'al_layer':"AL",
        'coral':"CORAL",
        'erm':"ERM",
        'irm':'IRM',
        'dro':'GroupDRO'
    },
    inplace=True
)

df_results['Task'].replace(
    {
        'longlos':'Long LOS',
        'sepsis':'Sepsis',
        'mortality':'Mortality',
        'invasivevent':'Invasive Ventilation',
    },
    inplace=True
)

# pivot tables
# compare with base
df_results = df_results.query(
    "Metric==['AUROC','AUPRC','Calibration']"
).pivot(
    index=["Task","Metric"],columns=["Framework","Unlabeled OOD Samples","Method"],values=["Difference"]
)
df_results.fillna(" ",inplace = True)
df_results.columns = pd.MultiIndex.from_tuples([x[1:] for x in df_results.columns], names = ['Framework','Unlabelled OOD Samples','Method'])
df_results = df_results.reindex(labels = ['Long LOS','Sepsis','Mortality','Invasive Ventilation'], level=0)
df_results = df_results.reindex(labels = ['AUROC','AUPRC','Calibration'],level=1)

df_results = df_results[[
        ('Domain Generalization',    '',              'IRM'),
        ('Domain Generalization',    '',         'GroupDRO'),
        ('Domain Generalization',    '',               'AL'),
        ('Domain Generalization',    '',            'CORAL'),
        (    'Domain Adaptation',  100.0,               'AL'),
        (    'Domain Adaptation',  100.0,            'CORAL'),
        (    'Domain Adaptation',  500.0,               'AL'),
        (    'Domain Adaptation',  500.0,            'CORAL'),
        (    'Domain Adaptation', 1000.0,               'AL'),
        (    'Domain Adaptation', 1000.0,            'CORAL'),
        (    'Domain Adaptation', 1500.0,               'AL'),
        (    'Domain Adaptation', 1500.0,            'CORAL'),
    ]]

In [10]:
# results relative to ERM[08-16]
df_results

Framework                           Domain Generalization  \
Unlabelled OOD Samples                                      
Method                                                IRM   
Task                 Metric                                 
Long LOS             AUROC           0.000 (-0.002,0.002)   
                     AUPRC           0.003* (0.000,0.006)   
                     Calibration  -0.007* (-0.008,-0.006)   
Sepsis               AUROC        -0.004* (-0.007,-0.001)   
                     AUPRC          -0.002 (-0.005,0.002)   
                     Calibration     0.001 (-0.001,0.002)   
Mortality            AUROC           0.000 (-0.001,0.002)   
                     AUPRC           0.002 (-0.004,0.007)   
                     Calibration  -0.001* (-0.001,-0.000)   
Invasive Ventilation AUROC          -0.000 (-0.003,0.002)   
                     AUPRC          -0.002 (-0.009,0.006)   
                     Calibration     0.000 (-0.001,0.001)   

Framework                                                  \
Unlabelled OOD Samples                                      
Method                                           GroupDRO   
Task                 Metric                                 
Long LOS             AUROC           0.002 (-0.001,0.004)   
                     AUPRC           0.001 (-0.002,0.005)   
                     Calibration     0.016* (0.014,0.017)   
Sepsis               AUROC        -0.030* (-0.036,-0.025)   
                     AUPRC        -0.026* (-0.031,-0.022)   
                     Calibration     0.011* (0.009,0.013)   
Mortality            AUROC        -0.050* (-0.055,-0.045)   
                     AUPRC        -0.042* (-0.051,-0.033)   
                     Calibration     0.026* (0.024,0.027)   
Invasive Ventilation AUROC          -0.001 (-0.004,0.001)   
                     AUPRC        -0.008* (-0.015,-0.001)   
                     Calibration     0.002 (-0.000,0.004)   

Framework                                                \
Unlabelled OOD Samples                                    
Method                                               AL   
Task                 Metric                               
Long LOS             AUROC         0.001 (-0.001,0.003)   
                     AUPRC         0.002 (-0.001,0.005)   
                     Calibration  -0.001 (-0.002,0.000)   
Sepsis               AUROC        -0.001 (-0.004,0.002)   
                     AUPRC        -0.002 (-0.005,0.001)   
                     Calibration  -0.001 (-0.002,0.000)   
Mortality            AUROC        -0.001 (-0.003,0.001)   
                     AUPRC         0.000 (-0.005,0.005)   
                     Calibration   0.000 (-0.001,0.001)   
Invasive Ventilation AUROC        -0.001 (-0.003,0.001)   
                     AUPRC        -0.005 (-0.012,0.001)   
                     Calibration  -0.000 (-0.001,0.001)   

Framework                                                  \
Unlabelled OOD Samples                                      
Method                                              CORAL   
Task                 Metric                                 
Long LOS             AUROC           0.000 (-0.002,0.002)   
                     AUPRC           0.002 (-0.001,0.005)   
                     Calibration     0.000 (-0.001,0.001)   
Sepsis               AUROC        -0.003* (-0.006,-0.000)   
                     AUPRC          -0.003 (-0.006,0.001)   
                     Calibration  -0.002* (-0.003,-0.000)   
Mortality            AUROC          -0.001 (-0.003,0.000)   
                     AUPRC          -0.000 (-0.005,0.005)   
                     Calibration     0.000 (-0.000,0.001)   
Invasive Ventilation AUROC          -0.000 (-0.002,0.002)   
                     AUPRC           0.002 (-0.004,0.009)   
                     Calibration     0.000 (-0.001,0.001)   

Framework                               Domain Adaptation  \
Unlabelled OOD Samples                                100   
Me

In [11]:
# baseline stats table
import docx
fpath = f"{tables_fpath}/eTabl_stats_rel_to_erm.docx"
doc = docx.Document(fpath)
t = doc.add_table(df_results.shape[0], df_results.shape[1])
for i in range(df_results.shape[0]):
    for j in range(df_results.shape[-1]):
        t.cell(i,j).text = str(df_results.values[i,j])
doc.save(fpath)